**Simple CNN**

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Loading Data
train_data = pd.read_csv('/content/fashion-mnist_test.csv')
test_data = pd.read_csv('/content/fashion-mnist_test.csv')

# Assuming the last column is the target and the rest are features
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

# Reshape data
X_train = X_train.reshape(-1, 28, 28, 1)  # Add a dimension for the channel
X_test = X_test.reshape(-1, 28, 28, 1)  # Add a dimension for the channel

num_classes = len(np.unique(y_train))  # Calculate the number of unique classes

# Create model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Use 'num_classes' here

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
from keras.utils import to_categorical

# One-hot encode target variable
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Now you can one-hot encode your labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_train.shape[1]  # Calculate the number of unique classes


# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: ", accuracy)



313/313 [==============================] - 2s 5ms/step - loss: 23.5306 - accuracy: 0.0499
Accuracy:  0.04989999905228615


**Simple CNN with data augmentation**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(X_train)

# fits the model on batches with real-time data augmentation
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=len(X_train) / 32, epochs=10)


Epoch 1/10
312/312 [==============================] - 9s 26ms/step - loss: 1.3324 - accuracy: 0.9905
Epoch 2/10
312/312 [==============================] - 9s 28ms/step - loss: 0.0994 - accuracy: 0.9933
Epoch 3/10
312/312 [==============================] - 10s 30ms/step - loss: 0.0478 - accuracy: 0.9950
Epoch 4/10
312/312 [==============================] - 10s 31ms/step - loss: 0.0394 - accuracy: 0.9959
Epoch 5/10
312/312 [==============================] - 9s 30ms/step - loss: 0.0290 - accuracy: 0.9961
Epoch 6/10
312/312 [==============================] - 10s 31ms/step - loss: 0.0313 - accuracy: 0.9958
Epoch 7/10
312/312 [==============================] - 8s 27ms/step - loss: 0.0242 - accuracy: 0.9963
Epoch 8/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0327 - accuracy: 0.9961
Epoch 9/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0263 - accuracy: 0.9962
Epoch 10/10
312/312 [==============================] - 8s 25ms/step - loss: 0.0315 - acc

**Pretrained with feature extraction (with data augmentation)**

In [ ]:
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=len(X_train) / 32, epochs=10)


Epoch 1/10
312/312 [==============================] - 8s 26ms/step - loss: 0.0243 - accuracy: 0.9965
Epoch 2/10
312/312 [==============================] - 9s 30ms/step - loss: 0.0263 - accuracy: 0.9965
Epoch 3/10
312/312 [==============================] - 9s 30ms/step - loss: 0.0282 - accuracy: 0.9961
Epoch 4/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0250 - accuracy: 0.9965
Epoch 5/10
312/312 [==============================] - 8s 25ms/step - loss: 0.0242 - accuracy: 0.9963
Epoch 6/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0244 - accuracy: 0.9966
Epoch 7/10
312/312 [==============================] - 9s 30ms/step - loss: 0.0256 - accuracy: 0.9964
Epoch 8/10
312/312 [==============================] - 8s 26ms/step - loss: 0.0243 - accuracy: 0.9964
Epoch 9/10
312/312 [==============================] - 8s 26ms/step - loss: 0.0220 - accuracy: 0.9963
Epoch 10/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0251 - accura

**Pretrained with last 5 layers Fine tuning (with as is data)**

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

conv_base.trainable = True

for layer in conv_base.layers[:-5]:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
313/313 [==============================] - 8s 22ms/step - loss: 0.0304 - accuracy: 0.9964 - val_loss: 0.0242 - val_accuracy: 0.9965
Epoch 2/10
313/313 [==============================] - 8s 25ms/step - loss: 0.0357 - accuracy: 0.9965 - val_loss: 0.0103 - val_accuracy: 0.9978
Epoch 3/10
313/313 [==============================] - 9s 28ms/step - loss: 0.0127 - accuracy: 0.9971 - val_loss: 0.0049 - val_accuracy: 0.9987
Epoch 4/10
313/313 [==============================] - 7s 21ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 0.0030 - val_accuracy: 0.9993
Epoch 5/10
313/313 [==============================] - 8s 25ms/step - loss: 0.0062 - accuracy: 0.9986 - val_loss: 0.0443 - val_accuracy: 0.9885
Epoch 6/10
313/313 [==============================] - 7s 21ms/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 7.5202e-04 - val_accuracy: 0.9999
Epoch 7/10
313/313 [==============================] - 8s 25ms/step - loss

**Pretrained with last 5 layers Fine tuning (with data augmentation)**

In [ ]:
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=len(X_train) / 32, epochs=10)
train_data = pd.read_csv('/content/fashion-mnist_test.csv')
test_data = pd.read_csv('/content/fashion-mnist_test.csv')

Epoch 1/10
312/312 [==============================] - 10s 29ms/step - loss: 0.0445 - accuracy: 0.9961
Epoch 2/10
312/312 [==============================] - 8s 26ms/step - loss: 0.0262 - accuracy: 0.9964
Epoch 3/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0235 - accuracy: 0.9963
Epoch 4/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0236 - accuracy: 0.9963
Epoch 5/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0244 - accuracy: 0.9964
Epoch 6/10
312/312 [==============================] - 8s 25ms/step - loss: 0.0216 - accuracy: 0.9965
Epoch 7/10
312/312 [==============================] - 8s 25ms/step - loss: 0.1916 - accuracy: 0.9958
Epoch 8/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0236 - accuracy: 0.9965
Epoch 9/10
312/312 [==============================] - 9s 29ms/step - loss: 0.0224 - accuracy: 0.9964
Epoch 10/10
312/312 [==============================] - 8s 27ms/step - loss: 0.0251 - accur